# quickstart

This quickstart example will put up a model server which uses a CNN model trained on CIFAR10 dataset to classify images.

## Create the model

To begin, clone the `spell-examples` repository. Then, use the `cifar10_cnn.py` script to train a CNN save it to SpellFS.

In [ ]:
%%bash
git clone https://github.com/spellml/examples.git
cd examples/keras
spell run \
    --machine-type k80 \
    --framework tensorflow \
    -- \
    python cifar10_cnn.py \
        --epochs 25 \
        --conv2_filter 65 \
        --conv2_kernel 7 \
        --dense_layer 621 \
        --dropout_3 0.0364

These parameters came from a hyperparameter search which was moderately successful. Whatever parameters you chose, it should produce a file at `runs/<RUN ID>/keras/saved_models/keras_cifar10_trained_model.h5` on SpellFS.

Finally, we can convert this file into a Spell Model named `cifar10` with the version name `example` from the output of the run.


In [ ]:
spell model create \
    --file keras/saved_models/keras_cifar10_trained_model.h5:model.h5 \
    cifar10:example runs/<RUN_ID>

The `--file` argument instructs that the file at `keras/saved_models/keras_cifar10_trained_model.h5` should be renamed `model.h5` in this model.

## Deploy the model server

The code used to load and serve this model is contained in `modelservers/cifar/predictor.py` in the `spell-examples` repository. We can create and start a model server with the `cifar10` model with the `spell server serve` command.

In [ ]:
spell server serve cifar10:example predictor.py

This will create a Kubernetes deployment. Before you can run inference on the model, it needs to initialize. You can check the status of a server and find its URL using the `spell server describe` command.

In [ ]:
spell server describe cifar10

Once the model server is running, you can test your model server either through issuing a cURL
command using the URL given by the `spell server describe` command, or by using the `spell server predict` command. For this example, we have provided a script at `modelservers/cifar/query_server.py` directory of the `spell-examples` repository which takes a path to an image and the URL of the CIFAR model server, base64-encodes the image, and calls its predict endpoint.

In [ ]:
python query_server.py http://url.to.model/server/predict path/to/image